In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#get links to each club
def getAllClubLinks(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    team_table = soup.select("table.stats_table")[0]
    team_links = team_table.find_all("a")
    team_links = [l.get("href") for l in team_links]
    team_links = [l for l in team_links if "/squads/" in l]
    team_links = [f"https://fbref.com{l}" for l in team_links]
    return team_links

In [3]:
def getClubData(requestData, param, match, keepList):
    soup = BeautifulSoup(requestData.text)
    all_links = soup.find_all("a")
    links = [l.get("href") for l in all_links]
    links = [l for l in links if l and param in l]
    links = "https://fbref.com" + links[0]
    shooting = requests.get(links)
    shooting = pd.read_html(shooting.text, match=match)[0]
    shooting.columns = shooting.columns.droplevel()
    shooting = shooting[keepList]
    return shooting[:-1]

In [4]:
def getAllClubData(url):
    allClubLinks = getAllClubLinks(url)
    names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
    
    allClubData = []
    for link in allClubLinks:
        club = requests.get(link)
        clubName = allClubLinks[1].split("/")[-1].replace("-Stats","").replace("-", " ")
        
        #get Scores
        scores = pd.read_html(club.text, match = "Scores & Fixtures")[0]
        scores = scores[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']]
        
        #get shooting and other stat
        shooting = getClubData(club, "/all_comps/shooting/", "Shooting", ['Date', 'Sh', "SoT"])
        misc = getClubData(club, "/all_comps/misc/", "Miscellaneous", ['Date', 'CrdY', 'CrdR', 'Fls', 'Off'])
        finalClubData = scores.merge(shooting,how ='left').merge(misc, how='left')
        finalClubData.rename(columns=names, inplace=True)
        finalClubData = finalClubData[finalClubData["Competition"] == "Premier League"]
        finalClubData["Team"] = clubName
        allClubData.append(finalClubData)
    return allClubData
    
    
    
url = "https://fbref.com/en/comps/9/Premier-League-Stats"
getAllClubData(url)

IndexError: list index out of range

In [ ]:

# url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# allClubLinks = getClubLinks(url)

# #chose one from all the links
# club = requests.get(allClubLinks[1])
# club_name = allClubLinks[1].split("/")[-1].replace("-Stats","").replace("-", " ")

# #thereisonyl one element
# matches = pd.read_html(club.text, match = "Scores & Fixtures")[0]
# matches.columns
# matches = matches[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA','Opponent', 'Poss']]


# shooting = getData("/all_comps/shooting/", "Shooting", ['Date', 'Sh', "SoT"])
# misc = getData("/all_comps/misc/", "Miscellaneous", ['Date', 'CrdY', 'CrdR', 'Fls', 'Off'])

# final_data = matches.merge(shooting,how ='left').merge(misc, how='left')
# names = {"Comp": "Competition", "Poss": "Possession", "Sh":"Shots", "SoT":"Shots Target", "CrdY":"Yellow", "CrdR":"Red", "Fls":"Fouls","Off":"Offside"}
# final_data.rename(columns=names, inplace=True)
# final_data
# final_data = final_data[final_data["Competition"] == "Premier League"]
# final_data["Team"] = club_name
# final_data

<!-- <a href="/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/keeper/Manchester-City-Match-Logs-All-Competitions">Goalkeeping</a>  /

<a href="/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions">Passing</a>

<a href="/en/squads/b8fd03ef/2022-2023/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions">Shooting</a>
 -->